In [1]:
%matplotlib widget

import scipp as sc
import plopp as pp
from plopp import widgets
import ipywidgets as ipw
from scipp.scipy.ndimage import gaussian_filter

In [2]:
da = sc.io.load_hdf5('nyc_taxi_data_2015_small.h5')
data = da.group("dropoff_hour").hist(dropoff_latitude=500, dropoff_longitude=500)
data

<scipp.DataArray>
Dimensions: Sizes[dropoff_hour:24, dropoff_latitude:500, dropoff_longitude:500, ]
Coordinates:
* dropoff_hour                int64  [dimensionless]  (dropoff_hour)  [0, 1, ..., 22, 23]
* dropoff_latitude          float64            [deg]  (dropoff_latitude [bin-edge])  [40.595, 40.5956, ..., 40.9143, 40.915]
* dropoff_longitude         float64            [deg]  (dropoff_longitude [bin-edge])  [-74.05, -74.0494, ..., -73.7306, -73.73]
Data:
                            float64         [counts]  (dropoff_hour, dropoff_latitude, dropoff_longitude)  [0, 0, ..., 0, 0]

In [3]:
# Slider node that provides index to slice
slider = ipw.IntSlider(description="Hour:", min=0, max=23)
slider_node = pp.widget_node(slider)

# Node that actually does the slicing
slice_node = pp.Node(lambda da, ind: da["dropoff_hour", ind], da=data, ind=slider_node)

# Add a 2D figure to show the NYC map
fig2d = pp.imagefigure(slice_node, norm="log", cbar=False)

# Add a node after the slicing to sum along the latitude dimension
sum_lat = pp.Node(sc.sum, slice_node, dim="dropoff_latitude")
# Add a node after the slicing to sum along the longitude dimension
sum_lon = pp.Node(sc.sum, slice_node, dim="dropoff_longitude")

# Add a node after the sum that performs as Gaussian smoothing
smooth = pp.Node(gaussian_filter, sum_lat, sigma=5)

# Add a 1D figure that will display sum along longitude
fig_lon = pp.linefigure(sum_lon, norm="log")
# Add a 1D figure that will display both raw latitude sum and smoothed data
fig_lat = pp.linefigure(sum_lat, smooth, norm="log")

widgets.Box([slider, fig2d, fig_lon, fig_lat])  # Container box

Box(children=(IntSlider(value=0, description='Hour:', max=23), InteractiveFig(children=(HBar(), HBox(children=…

In [6]:
# Slider node that provides index to slice
slider = ipw.IntSlider(description="Hour:", min=0, max=23)
slider_node = pp.widget_node(slider)

# Node that actually does the slicing
slice_node = pp.Node(lambda da, ind: da["dropoff_hour", ind], da=data, ind=slider_node)

# Add a 2D figure to show the NYC map
fig2d = pp.figure2d(slice_node, norm="log", cbar=False)

# Add a node after the slicing to sum along the latitude dimension
sum_lat = pp.Node(sc.sum, slice_node, dim="dropoff_latitude")
# Add a node after the slicing to sum along the longitude dimension
sum_lon = pp.Node(sc.sum, slice_node, dim="dropoff_longitude")


# Add a new slider that will act as input to the Gaussian smoothing node
smooth_slider = ipw.IntSlider(description="kernel:", min=1, max=25)
smooth_slider_node = pp.widget_node(smooth_slider)


# Add a node after the sum that performs as Gaussian smoothing
smooth = pp.Node(gaussian_filter, sum_lat, sigma=smooth_slider_node)

# Add a 1D figure that will display sum along longitude
fig_lon = pp.figure1d(sum_lon, norm="log")
# Add a 1D figure that will display both raw sum and smoothed data
fig_lat = pp.figure1d(sum_lat, smooth, norm="log")

widgets.Box([[slider, smooth_slider], [fig2d, fig_lon], fig_lat])  # Container box

/var/folders/dw/4qtkldcj3xn4dmlgmmzkjbs00000gp/T/ipykernel_94649/561226521.py:9: VisibleDeprecationWarning: figure2d is deprecated. Use ``imagefigure`` instead.
  fig2d = pp.figure2d(slice_node, norm="log", cbar=False)
/var/folders/dw/4qtkldcj3xn4dmlgmmzkjbs00000gp/T/ipykernel_94649/561226521.py:26: VisibleDeprecationWarning: figure1d is deprecated. Use ``linefigure`` instead.
  fig_lon = pp.figure1d(sum_lon, norm="log")
/var/folders/dw/4qtkldcj3xn4dmlgmmzkjbs00000gp/T/ipykernel_94649/561226521.py:28: VisibleDeprecationWarning: figure1d is deprecated. Use ``linefigure`` instead.
  fig_lat = pp.figure1d(sum_lat, smooth, norm="log")


Box(children=(HBar(children=(IntSlider(value=0, description='Hour:', max=23), IntSlider(value=1, description='…